<a href="https://colab.research.google.com/github/kokutoubanira/NLP_Project_sample/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [60]:
import torch 
from torch import nn,optim
from torch.utils.data import(Dataset, DataLoader, TensorDataset)
import torch.nn.functional as F
import tqdm
import re 
import collections
import itertools
import MeCab
import neologdn
import math
import emoji
import matplotlib.pyplot as plt

import math
import torchtext
from torchtext.vocab import Vectors
import glob
import os
import io
import string
import re
import random
from time import sleep
import csv
import pandas as pd
import torchnlp.metrics.bleu as bleu

mecab = MeCab.Tagger('-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')
mecab.parse('')  # バグ対処

'EOS\n'

In [66]:
remove_marks_regex = re.compile("[\,\(\)\[\]\*:;]|<.*?>")
shift_marks_regex = re.compile("([?!\.])")



unk = 0
sos = 1
eos = 2

def normalize(text):
    text = text.lower()
#     #不要な文字を削除
#     text = remove_marks_regex.sub("", text)
    #?!.と単語の間に空白を挿入
    text = shift_marks_regex.sub(r"\1", text)
    #重ね表現の削除
    text = neologdn.normalize(text)
    #url削除
    text = re.sub(r'https?://[\w/:%#\$&\?\(\)~\.=\+\-]+', '', text)
    #絵文字削除
    text = ''.join(['' if c in emoji.UNICODE_EMOJI else c for c in text])
    #桁区切りの削除
    text = re.sub(r'(\d)([,.])(\d+)', r'\1\3', text)
    text = re.sub(r'\d+', '0', text)
    # 半角記号の置換
    text = re.sub(r'[!-/:-@[-`{-~]', r' ', text)
    # 全角記号の置換 (ここでは0x25A0 - 0x266Fのブロックのみを除去)
    text = re.sub(u'[■-♯]', ' ', text)
    text = text.replace("【", " ").replace("】", " ").replace("『"," ").replace("』", " ").replace("、", " ").replace("。", " ").replace("”", " ").replace('"', " ")
    return text

def make_wakati(sentence):
    sentence = sentence.lower()
    # MeCabで分かち書き
    sentence = re.sub(r'[0-9０-９a-zA-Zａ-ｚＡ-Ｚ]+', " ", sentence)
    # 記号もろもろ除去
    sentence = re.sub(r'[\．_－―─！＠＃＄％＾＆\-‐|\\＊\“（）＿■×+α※÷⇒—●★☆〇◎◆▼◇△□(：〜～＋=)／*&^%$#@!~`){}［］…\[\]\"\'\”\’:;<>?＜＞〔〕〈〉？、。・,\./『』【】「」→←○《》≪≫\n\u3000]+', "", sentence)
    #sentence = mecab.parse(sentence)
    # 半角全角英数字除去
    parsed_lines = mecab.parse(sentence).split("\n")[:-2]
    surfaces = [l.split('\t')[0] for l in parsed_lines]
    features = [l.split('\t')[1] for l in parsed_lines]
    bases = [f.split(',')[6] for f in features]
    # 品詞を取得
    pos = [f.split(',')[0] for f in features]
    # 各単語を原型に変換する
    token_list = [b if b != '*' else s for s, b in zip(surfaces, bases)]
    # 名詞,動詞,形容詞のみに絞り込み
    target_pos = ['名詞', '形容詞', '動詞']
    token_list = [t for t, p in zip(token_list, pos) if p in target_pos]

    # スペースで区切って形態素の配列へ
    #wakati = sentence.split(" ")
    # 空の要素は削除
    wakati = list(filter(("").__ne__, token_list))
    return wakati

def parse_line(src):
    #翻訳元と翻訳先それぞれのトークンリストを作成する
    src = mecab.parse(str(src))
    src = make_wakati(str(src))
    return src

def build_vocab(tokens):
    #ファイル中のすべての文章でのトークン数を数える
    counts = collections.Counter(tokens)
    #トークンの出現数の多い順に並べる
    sorted_counts = sorted(counts.items(), key=lambda c: c[1], reverse=True)
    #3つのタグを追加して正引きリストと逆引き用辞書を作る
    word_list = ["<UNK>", "<SOS>", "<EOS>"] + [x[0] for x in sorted_counts]
    word_dict = dict((w, i) for i, w in enumerate(word_list))
    return word_list, word_dict

def words2tensor(words, word_dict, max_len, padding=0):
    #末尾に終了タグをつける
    print(words)
    words = words + ["<EOS>"]
    #辞書を利用して数値のリストに変換する
    words = [word_dict.get(w,0) for w in words]
    seq_len = len(words)
    #長さがmax_len以下の場合はパディングする
    if seq_len < max_len + 1:
        words = words + [padding] * (max_len + 1 - seq_len)
    #Tensorに変換して返す
    return torch.tensor(words, dtype=torch.int64), seq_len

In [67]:
make_wakati("勾配ブースティング決定木(Gradient Boosting Decision Tree: GBDT)とは、「勾配降下法(Gradient)」と「アンサンブル学習(Boosting)」、「決定木(Decision Tree)」の3つの手法が組み合わされた機械学習の手法")

['勾配',
 'ブースティング',
 '決定',
 '木',
 '勾配',
 '降下',
 '法',
 'アンサンブル',
 '学習',
 '決定木',
 '手法',
 '組み合わす',
 'れる',
 '機械学習',
 '手法']

In [68]:
import pandas as pd
df = pd.read_csv("livedoor_train.csv")

In [69]:
src_vocab_list = []
for i in df["Text"]:
    src_vocab_list.append(parse_line(i))
#語彙集の作成
vocab_word_list, vocab_word_dict = build_vocab(itertools.chain.from_iterable(src_vocab_list))


In [45]:
src_data = [words2tensor(words, vocab_word_dict, 200) for words in src_vocab_list]

http://news.livedoor.com/article/detail/6700966/
2012-06-28T06:55:00+0900
NTTドコモ、MEDIAS Tab N-06Dに電源ON後に画面が表示されないなどの不具合でソフトウェア更新を提供開始
MEDIAS Tab N-06Dにソフトウェア更新！ 

NTTドコモは27日、NOTTVを視聴できるモバキャス対応やおサイフケータイ、赤外線、通話機能にも対応した7インチサイズAndroidタブレット「MEDIAS Tab N-06D」（NEC製）において電源ON後に画面が表示されないなどの不具合が見つかったとしてネットワーク経由による本体ファームウェアアップデートサービス「ソフトウェア更新」を提供開始したことをお知らせしています。

今回の更新で修正される不具合は以下の2点です。MEDIAS Tab N-06Dのソフトウェア更新は今回が2回目で、更新にかかる時間は約6分。更新期間は2015年6月30日まで。

※下記以外にも、より快適にご利用いただくための更新や一部仕様および表示変更などが含まれています。

改善される事象
・microSDXCカードを差し込むと、microSDXCカード内のデータが破損される。
・電源をOFFにした際、まれに次回電源をONにしても画面が真っ暗のままで表示されない場合がある。



更新は、自動更新に対応しており、新しいソフトウェアを自動でダウンロードし、あらかじめ設定した時間（デフォルトは午前2時〜午前4時）に書換えが行われます。また、ホーム画面にて「メニューボタン」⇒「設定」⇒「端末情報」⇒「ソフトウェア更新」を選択し、画面の案内に従って操作を行うことで、即時更新も可能となっています。

詳しい操作手順は「ソフトウェア更新 簡易操作手順（MEDIAS TAB N-06D）（PDF形式：693KB）」を参照して下さい。

ソフトウェア更新を行う際には、以下の注意点がありますので、ご注意ください。

・ソフトウェア更新はお客様の責任において実施してください。
・ソフトウェア更新は手順をよくお読みいただき実施してください。
・ソフトウェア更新はデータを残したまま行うことができますが、万が一に備え、大切なデータは必ずバックアップを行ってから、実施してください。バックアップ

TypeError: must be str, not list

In [3]:
class Embedder(nn.Module):
    '''idで示されている単語をベクトルに変換します'''
    def __init__(self, text_embedding_vectors):
        super(Embedder, self).__init__()

        self.embeddings = nn.Embedding(text_embedding_vectors, 512)
    def forward(self, x):
        x_vec = self.embeddings(x)
        return x_vec
class PositionalEncoder(nn.Module):
    '''入力された単語の位置を示すベクトル情報を付加する'''

    def __init__(self, d_model=300, max_seq_len=256):
        super().__init__()

        self.d_model = d_model  # 単語ベクトルの次元数

        # 単語の順番（pos）と埋め込みベクトルの次元の位置（i）によって一意に定まる値の表をpeとして作成
        pe = torch.zeros(max_seq_len, d_model)

        # GPUが使える場合はGPUへ送る
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        pe = pe.to(device)

        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos /
                                          (10000 ** ((2 * (i + 1))/d_model)))

        # 表peの先頭に、ミニバッチ次元となる次元を足す
        self.pe = pe.unsqueeze(0)

        # 勾配を計算しないようにする
        self.pe.requires_grad = False

    def forward(self, x):

        # 入力xとPositonal Encodingを足し算する
        # xがpeよりも小さいので、大きくする
        ret = math.sqrt(self.d_model)*x + self.pe
        return ret



In [29]:
class PositionalEncoder(nn.Module):
    '''入力された単語の位置を示すベクトル情報を付加する'''

    def __init__(self, d_model=300, max_seq_len=256):
        super().__init__()

        self.d_model = d_model  # 単語ベクトルの次元数

        # 単語の順番（pos）と埋め込みベクトルの次元の位置（i）によって一意に定まる値の表をpeとして作成
        pe = torch.zeros(max_seq_len, d_model)

        # GPUが使える場合はGPUへ送る
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        pe = pe.to(device)

        for pos in range(max_seq_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2 * i)/d_model)))
                pe[pos, i + 1] = math.cos(pos /
                                          (10000 ** ((2 * (i + 1))/d_model)))

        # 表peの先頭に、ミニバッチ次元となる次元を足す
        self.pe = pe.unsqueeze(0)

        # 勾配を計算しないようにする
        self.pe.requires_grad = False

    def forward(self, x):

        # 入力xとPositonal Encodingを足し算する
        # xがpeよりも小さいので、大きくする
        ret = math.sqrt(self.d_model)*x + self.pe
        return ret

def Attention(q, k, v, d_k, mask=None, dropout=None):
    
    scores = torch.matmul(q, k.transpose(-2, -1)) / math.sqrt(d_k)
    if mask is not None:
        mask = mask.unsqueeze(1)
        scores = scores.masked_fill(mask == 0, -1e9)
    scores = F.softmax(scores, dim=-1)
    
    if dropout is not None:
        scores = dropout(scores)
        
    output = torch.matmul(scores, v)
    return output

class MultiHeadAttention(nn.Module):

    def __init__(self, d_model=300):
        super().__init__()

        #全結合層で特徴量を変換する
        self.q_linear = nn.Linear(d_model, d_model)
        self.v_linear = nn.Linear(d_model, d_model)
        self.k_linear = nn.Linear(d_model, d_model)
        
        # 出力時に使用する全結合層
        self.out = nn.Linear(d_model, d_model)

        # Attentionの大きさ調整の変数
        self.d_k = d_model
        
        # multi head Attention用
#         self.d_k = d_model / heads
#         self.h = heads

    def forward(self, q, k, v, mask):
        # 全結合層で特徴量を変換
        k = self.k_linear(k)
        q = self.q_linear(q)
        v = self.v_linear(v)
        # Attentionの値を計算する
        # 各値を足し算すると大きくなりすぎるので、root(d_k)で割って調整
        weights = torch.matmul(q, k.transpose(1, 2)) / math.sqrt(self.d_k)
        # ここでmaskを計算
        mask = mask.unsqueeze(1)
        weights = weights.masked_fill(mask == 0, -1e9)
        # softmaxで規格化をする
        normlized_weights = F.softmax(weights, dim=-1)
        # AttentionをValueとかけ算
        output = torch.matmul(normlized_weights, v)
        # 全結合層で特徴量を変換
        output = self.out(output)
        return output, normlized_weights

class FeedForward(nn.Module):
    def __init__(self, d_model, d_ff=1024, dropout=0.1):
        '''Attention層から出力を単純に全結合層2つで特徴量を変換するだけのユニットです'''
        super().__init__()

        self.linear_1 = nn.Linear(d_model, d_ff)
        self.dropout = nn.Dropout(dropout)
        self.linear_2 = nn.Linear(d_ff, d_model)

    def forward(self, x):
        x = self.linear_1(x)
        x = self.dropout(F.relu(x))
        x = self.linear_2(x)
        return x
class TransformerBlock(nn.Module):
    def __init__(self, d_model, dropout=0.1):
        super().__init__()

        # LayerNormalization層
        # https://pytorch.org/docs/stable/nn.html?highlight=layernorm
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)

        # Attention層
        self.attn = MultiHeadAttention(d_model)

        # Attentionのあとの全結合層2つ
        self.ff = FeedForward(d_model)

        # Dropout
        self.dropout_1 = nn.Dropout(dropout)
        self.dropout_2 = nn.Dropout(dropout)

    def forward(self, x, mask):
        # 正規化とAttention
        x_normlized = self.norm_1(x)
        output, normlized_weights = self.attn(
            x_normlized, x_normlized, x_normlized, mask)

        x2 = x + self.dropout_1(output)

        # 正規化と全結合層
        x_normlized2 = self.norm_2(x2)
        output = x2 + self.dropout_2(self.ff(x_normlized2))

        return output, normlized_weights
class ClassificationHead(nn.Module):
    '''Transformer_Blockの出力を使用し、最後にクラス分類させる'''

    def __init__(self, d_model=300, output_dim=2):
        super().__init__()

        
        # 全結合層
        self.linear = nn.Linear(d_model, output_dim)  # output_dimはポジ・ネガの2つ

        # 重み初期化処理
        nn.init.normal_(self.linear.weight, std=0.02)
        nn.init.normal_(self.linear.bias, 0)

    def forward(self, x):

#         x0 = x[:, 0, :]  # 各ミニバッチの各文の先頭の単語の特徴量（300次元）を取り出す
        out = self.linear(x)
        return out
        
class TransformerClassification(nn.Module):
    '''Transformerでクラス分類させる'''

    def __init__(self, text_embedding_vectors, d_model=300, max_seq_len=256, output_dim=2):
        super().__init__()

        # モデル構築
        self.net1 = Embedder(text_embedding_vectors)
        self.net2 = PositionalEncoder(d_model=d_model, max_seq_len=max_seq_len)
        self.net3_1 = TransformerBlock(d_model=d_model)
        self.net3_2 = TransformerBlock(d_model=d_model)
        self.net4 = ClassificationHead(output_dim=output_dim, d_model=d_model)


    def forward(self, x, mask):
        x1 = self.net1(x)  # 単語をベクトルに
        x2 = self.net2(x1)  # Positon情報を足し算
        x3_1, normlized_weights_1 = self.net3_1(x2, mask)  # Self-Attentionで特徴量を変換
        x3_2, normlized_weights_2 = self.net3_2(x3_1, mask)  # Self-Attentionで特徴量を変換
        x0 = x3_2[:, 0, :]
        x4 = self.net4(x0)  # 最終出力の0単語目を使用して、分類0-1のスカラーを出力
  
        return x4, normlized_weights_1, normlized_weights_2

In [72]:
print(torch.cuda.device_count())

1


In [73]:
def weights_init(m):
    classname = m.__class__.__name__
    if classname.find('Liner') != -1:
        #Liner層初期化
        nn.init.kaiming_normal_(m.weights)
        if m.bias is not None:
            nn.init.constant_(m.bias, 0.0)

In [74]:
class Embedder(nn.Module):
    '''idで示されている単語をベクトルに変換します'''

    def __init__(self, text_embedding_vectors):
        super(Embedder, self).__init__()

        self.embeddings = nn.Embedding(text_embedding_vectors, 512)
    def forward(self, x):
        x_vec = self.embeddings(x)

        return x_vec

In [110]:
# ミニバッチの用意
batch = next(iter(test_dl))
x = batch.Text[0]
# 入出力
input_pad = 1 
input_mask = (x != input_pad)
out, normlized_weights_1, normlized_weights_2 = net(x.to('cuda'), input_mask.to("cuda"))

ValueError: too many dimensions 'str'

In [119]:
import tqdm
from statistics import mean
net = TransformerClassification(text_embedding_vectors=len(TEXT.vocab.stoi.values()), d_model=512, max_seq_len=100, output_dim=9)
net.train()
#Transformerblockモジュールを初期化
net.net3_1.apply(weights_init)
net.net3_2.apply(weights_init)
print("ネットワーク設定完了")

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

def train_model(net, dataloders_dict, criterion, optimizer, num_epochs):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print("使用デバイス：", device)
    print('-----start-------')
    # ネットワークをGPUへ
    net = net.to(device)

    # ネットワークがある程度固定であれば、高速化させる
    torch.backends.cudnn.benchmark = True
    total_loss = []
    # epochのループ
    for epoch in range(num_epochs):
        # epochごとの訓練と検証のループ
        for phase in ['train']:
            if phase == 'train':
                net.train()  # モデルを訓練モードに

            epoch_loss = 0.0  # epochの損失和
            epoch_corrects = 0  # epochの正解数
            losses = []
            # データローダーからミニバッチを取り出すループ
            for batch in tqdm.tqdm(dataloders_dict[phase]):
                
                loss = 0
                # batchはTextとLableの辞書オブジェクト
                # GPUが使えるならGPUにデータを送る
                inputs = batch.Text[0].to(device)  # 文章
                labels = batch.Label[0].to(device)  # ラベル

                # optimizerを初期化
                optimizer.zero_grad()

                # 順伝搬（forward）計算
                with torch.set_grad_enabled(phase == 'train'):

                    # mask作成
                    input_pad = 1  # 単語のIDにおいて、'<pad>': 1 なので
                    input_mask = (inputs != input_pad)

                    # Transformerに入力
                    outputs, _, _ = net(inputs, input_mask)
                    print(outputs.shape, labels.shape)
                    loss = criterion(outputs, labels)  # 損失を計算
                    #losses.append(loss.item())
                    _, preds = torch.max(outputs, 1)  # ラベルを予測

                    # 訓練時はバックプロパゲーション
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    
                    # 結果の計算
                    epoch_loss += loss.item() * inputs.size(0)  # lossの合計を更新
                    # 正解数の合計を更新
                    epoch_corrects += torch.sum(preds == labels.data)
            # epochごとのlossと正解率
            epoch_loss = epoch_loss / len(dataloders_dict[phase].dataset)
            epoch_acc = epoch_corrects.double(
            ) / len(dataloders_dict[phase].dataset)       
            print('Epoch {}/{} | {:^5} |  Loss: {:.4f} Acc: {:.4f}'.format(epoch+1, num_epochs,
                                                                           phase, epoch_loss, epoch_acc))
    return net

ネットワーク設定完了


In [120]:
num_epoch = 10
net_trained = train_model(net, dataloders_dict, criterion, optimizer, num_epochs=num_epoch)

# save_path = 'str(num_epoch)' + 'epoch.pth'
# torch.save(net_trained.state_dict(), save_path)

使用デバイス： cuda:0
-----start-------


ValueError: too many dimensions 'str'